In [11]:
# Solo la primera vez que se ejecuta
#!pip install pandas requests bs4 googletrans deep-translator nltk langchain
# !pip install --upgrade nltk

                                              0.0/2.0 MB ? eta -:--:--
     ---------------                          0.8/2.0 MB 16.8 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 25.2 MB/s eta 0:00:00
                                              0.0/45.3 kB ? eta -:--:--
     ---------------------------------------- 45.3/45.3 kB 2.2 MB/s eta 0:00:00
                                              0.0/395.8 kB ? eta -:--:--
     ------------------------------------- 395.8/395.8 kB 25.7 MB/s eta 0:00:00
                                              0.0/49.4 kB ? eta -:--:--
     ---------------------------------------- 49.4/49.4 kB ? eta 0:00:00
                                              0.0/2.0 MB ? eta -:--:--
     ---------------------------------------- 2.0/2.0 MB 63.7 MB/s eta 0:00:00
  Attempting uninstall: jsonpatch
    Found existing installation: jsonpatch 1.32
    Uninstalling jsonpatch-1.32:
      Successfully uninstalled jsonpatch-1.32


In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from requests.exceptions import ChunkedEncodingError
import re
import os
from deep_translator import GoogleTranslator
import glob

In [6]:
url = "https://ideasai.com/"  # Sustituye esto por la URL de la página que quieres scrapear
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


In [7]:
# Ahora puedes usar 'soup' para encontrar elementos en la página. Por ejemplo:
titulo = soup.find('title').text # Esto obtendrá el título de la página
titulo

'\n\t\t\t\tStartup Ideas powered by OpenAI\t\t\t'

In [8]:
# All .tag-link inside <div class="tag-container">
tags = soup.find('div', attrs={'class': 'tag-container'}).find_all('a', attrs={'class': 'tag-link'})

tags_list = []

for tag in tags:
    tags_list.append(tag.text)
    
    num_tags = len(tags_list)
    
print(f"El número de etiquetas es: {num_tags}")

El número de etiquetas es: 1637


In [9]:
# Enter in one of the tags and get h3 classname .idea
tag = tags_list[0]
tag_url = url + tag
tag_url

response = requests.get(tag_url)
soup = BeautifulSoup(response.text, 'html.parser')

ideas = soup.find_all('h3', attrs={'class': 'idea'})

ideas_list = []

for idea in ideas:
    ideas_list.append(idea.text)
    num_ideas = len(ideas_list)

print(f"El número de categorias de ideas es: {num_ideas}")

El número de ideas es: 211


In [10]:
# create an empty list to store the data
data = []

# iterate over all the tags
for i, tag in enumerate(tags_list, 1):
    try:
        tag_url = url + tag
        response = requests.get(tag_url)
        response.raise_for_status()  # verifica si hay errores en la respuesta HTTP
        soup = BeautifulSoup(response.text, 'html.parser')
        ideas = soup.find_all('h3', attrs={'class': 'idea'})
        for idea in ideas:
            data.append({'Tag': tag, 'Idea': idea.text})
        print(f"Enlaces procesados: {i}, Enlaces restantes: {len(tags_list) - i}")
    except ChunkedEncodingError as e:
        print(f"Error: {e}. Reintentando...")

# create a dataframe from the list
df = pd.DataFrame(data)

# save the results into a CSV file
df.to_csv('ideas.csv', index=False)

Enlaces procesados: 1, Enlaces restantes: 1636
Enlaces procesados: 2, Enlaces restantes: 1635
Enlaces procesados: 3, Enlaces restantes: 1634
Enlaces procesados: 4, Enlaces restantes: 1633
Enlaces procesados: 5, Enlaces restantes: 1632
Enlaces procesados: 6, Enlaces restantes: 1631
Enlaces procesados: 7, Enlaces restantes: 1630
Enlaces procesados: 8, Enlaces restantes: 1629
Enlaces procesados: 9, Enlaces restantes: 1628
Enlaces procesados: 10, Enlaces restantes: 1627
Enlaces procesados: 11, Enlaces restantes: 1626
Enlaces procesados: 12, Enlaces restantes: 1625
Enlaces procesados: 13, Enlaces restantes: 1624
Enlaces procesados: 14, Enlaces restantes: 1623
Enlaces procesados: 15, Enlaces restantes: 1622
Enlaces procesados: 16, Enlaces restantes: 1621
Enlaces procesados: 17, Enlaces restantes: 1620
Enlaces procesados: 18, Enlaces restantes: 1619
Enlaces procesados: 19, Enlaces restantes: 1618
Enlaces procesados: 20, Enlaces restantes: 1617
Enlaces procesados: 21, Enlaces restantes: 1616
E